In [1]:
'''
Regina Catipon
MACS 30100
2/29/2020
'''
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score 
from sklearn.linear_model import LogisticRegression, ElasticNetCV, ElasticNet 
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score

#from sklearn.inspection import plot_partial_dependence

# <span style="color: blue">Conceptual: Cost functions for classification trees

<span style="color: blue">1. (15 points) Consider the Gini index, classification error, and cross-entropy in simple classification settings with two classes. Of these three possible cost functions, which would be best to use when growing a decision tree? Which would be best to use when pruning a decision tree? Why?

Because classification error is simply the fraction of misclassified training observations it is not sensitive enough for tree growing. The other two impurity measures, the Gini index and cross-entropy are considered to be best because they control for variance across the classes which helps to optimize splitting for growth. For pruning, classifcation error is considered to be an appropriate measure because it prioritizes accuracy.


# Application: Predicting attitudes towards racist college professors

## Estimate the models
2. (35 points; 5 points/model) Estimate the following models, predicting colrac using the training set (the training .csv) with 10-fold CV:
    - Logistic regression
    - Naive Bayes
    - Elastic net regression
    - Decision tree (CART)
    - Bagging
    - Random forest
    - Boosting

Tune the relevant hyperparameters for each model as necessary. Only use the tuned model with the best performance for the remaining exercises. Be sure to leave sufficient time for hyperparameter tuning. Grid searches can be computationally taxing and take quite a while, especially for tree-aggregation methods.

In [5]:
#load data
train = pd.read_csv("./data/gss_train.csv")
test = pd.read_csv("./data/gss_test.csv")

In [14]:
#inspect
train.head()
#don't have to check for nulls because of preprocessinh

age                       0
attend                    0
authoritarianism          0
black                     0
born                      0
childs                    0
colath                    0
colrac                    0
colcom                    0
colmil                    0
colhomo                   0
colmslm                   0
con_govt                  0
egalit_scale              0
evangelical               0
grass                     0
happy                     0
hispanic_2                0
homosex                   0
income06                  0
mode                      0
owngun                    0
polviews                  0
pornlaw2                  0
pray                      0
pres08                    0
reborn_r                  0
science_quiz              0
sex                       0
sibs                      0
social_connect            0
south                     0
teensex                   0
tolerance                 0
tvhours                   0
vetyears            

In [16]:
# dropping colrac column and set predictors
X_train = train.drop(['colrac'], axis=1)
X_test = test.drop(['colrac'], axis=1)

#setting prediction output to colrac
y_train, y_test = train['colrac'], test['colrac']

In [21]:
#Logistic regression

log_reg = LogisticRegression()


In [31]:
#Naive Bayes
naive_bayes = GaussianNB()

#AUC
nb_score = cross_val_score(naive_bayes, X_train, 
                           y_train, scoring = 'roc_auc', cv = 10)
nb_roc = np.mean(nb_score)

#
nb_accu = cross_val_score(naive_bayes, X_train, 
                           y_train, scoring = 'accuracy', cv = 10)
nb_err = np.mean(nb_accu)

print("AUC/ROC", nb_roc)
print("Error", nb_err)

AUC/ROC 0.8080500250922787
Error 0.7344474902240962


In [26]:
#Elastic net regression

elas = ElasticNetCV(cv=10)
elas.fit(X_train, y_train)

# have to tune alpha and l1
print( elas.alpha_, elas.l1_ratio)


0.0038452641680228584 0.5


In [37]:
#using that alpha and l1 ratio
elas_best = ElasticNet(alpha = 0.0038452641680228584, l1_ratio=0.5)

# is MSE best?
elas_score = cross_val_score(elas_best, X_train, y_train,
                          scoring = 'neg_mean_squared_error', cv=10)
elas_mse = np.mean(elas_score)

#AUC/ROC
elas_score = cross_val_score(elas_best, X_train, y_train,
                          scoring = 'roc_auc', cv=10)
elas_roc = np.mean(elas_roc)

print("AUC/ROC", elas_roc)
print("Error", -1*elas_mse)

AUC/ROC 0.8740225489138439
Error 0.1471453221731916


In [20]:
#Decision tree (CART)
#Bagging
#Random forest

#Boosting

AUC/ROC 0.8080500250922787
Accuracy 0.7344474902240962


## Evaluate the models
3. (20 points) Compare and present each model's (training) performance based on
    - Cross-validated error rate
    - ROC/AUC

4. (15 points) Which is the best model? Defend your choice.

## Evaluate the best model
5. (15 points) Evaluate the final, best model's (selected in 4) performance on the test set (the test .csv) by calculating and presenting the classification error rate and AUC. Compared to the fit evaluated on the training set in questions 3-4, does the "best" model generalize well? Why or why not? How do you know?

## Bonus: PDPs/ICE
6. (Up to 5 extra points) Present and substantively interpret the "best" model (selected in question 4) using PDPs/ICE curves over the range of: tolerance and age. Note, interpretation must be more than simple presentation of plots/curves. You must sufficiently describe the changes in probability estimates over the range of these two features. You may earn up to 5 extra points, where partial credit is possible if the solution is insufficient along some dimension (e.g., technically/code, interpretation, visual presentation, etc.).